In [16]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"
from keras import backend as K
import tensorflow as tf
from keras.layers import Conv2D, Dropout, LSTM, BatchNormalization, Input,Activation, MaxPool2D, Flatten, Dense,TimeDistributed
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.layers.convolutional import ZeroPadding2D
from keras import metrics
import h5py
from sklearn.metrics import confusion_matrix
from data_utils import convert_to_one_hot, permute

In [ ]:
VIDEOS_DIR, IMAGES_DIR, classes, class_to_index, videos = get_global_variables()
print(VIDEOS_DIR)
print(IMAGES_DIR)
print(classes)
print(class_to_index)
print(videos)

In [ ]:
def end_to_end_gpu(input_shape):
    X_input = Input(input_shape)
    
    #with tf.device('/gpu:0'):     
    X = TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding = "same"), name ="Conv_2a")(X_input)
    X = TimeDistributed(MaxPool2D((2, 2)), name = "Pool_2")(X)
    X = TimeDistributed(Dropout(0.2), name= "Dropout_2")(X)

    X = TimeDistributed(Conv2D(32,(3,3), name='Conv_3a', activation='relu'))(X)
    X = TimeDistributed(MaxPool2D((2, 2), name = "Pool_3"))(X)
    X = TimeDistributed(Dropout(0.2), name = "Dropout_3")(X)

    X = TimeDistributed(Conv2D(8,(1,1), activation='relu'), name='Conv_1x1')(X)
    X = TimeDistributed(Flatten())(X)
    X = TimeDistributed(Dropout(0.3))(X)
    #Y = TimeDistributed(Dense(9,activation='softmax',name='final'),name='Secondary_Output')(X)

    X = LSTM(48, return_sequences=False,dropout=0.3, name = "LSTM1")(X)
    #X = LSTM(32, return_sequences=False)(X)
    X = Dense(9, activation='softmax',name='Primary_Output')(X)

    return Model(X_input, outputs=X)


In [2]:
#model = end_to_end_gpu((40, 22, 22, 16))
model = load_model('shizz_64_86.h5')

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 22, 22, 16)    0         
_________________________________________________________________
Conv_2a (TimeDistributed)    (None, 40, 22, 22, 32)    4640      
_________________________________________________________________
Pool_2 (TimeDistributed)     (None, 40, 11, 11, 32)    0         
_________________________________________________________________
Dropout_2 (TimeDistributed)  (None, 40, 11, 11, 32)    0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 40, 9, 9, 32)      9248      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 40, 4, 4, 32)      0         
_________________________________________________________________
Dropout_3 (TimeDistributed)  (None, 40, 4, 4, 32)      0         
__________

In [4]:
image_size=(172, 172)
max_len = 40
stride = 10
X_train = np.load('X_encoded.npy')
Y_train = np.load('../Numpy/End2End/Y_train_(172, 172)_40_10.npy')
X_test = np.load('X_test_encoded.npy')
Y_test = np.load('../Numpy/End2End/Y_test_(128, 128)_40_10.npy')
X_test_unseen = np.load('X_test_unseen_encoded.npy')
Y_test_unseen = np.load('../Numpy/End2End/Y_test_unseen_(172, 172)_40_10.npy')

In [ ]:
model.compile(loss='categorical_crossentropy',
        metrics=['accuracy'],
        optimizer='adam')

In [14]:
Y_train = convert_to_one_hot(Y_train, 9)

In [17]:
X_train, Y_train = permute(X_train, Y_train)

In [25]:
model.fit(X_train, Y_train, epochs=10, batch_size=120, validation_split=0.1)

Train on 582 samples, validate on 65 samples
Epoch 1/10
582/582 [==============================] - 28s 47ms/step - loss: 0.0540 - acc: 0.9914 - val_loss: 0.0170 - val_acc: 1.0000
Epoch 2/10
582/582 [==============================] - 31s 53ms/step - loss: 0.0499 - acc: 0.9897 - val_loss: 0.0190 - val_acc: 1.0000
Epoch 3/10
582/582 [==============================] - 36s 62ms/step - loss: 0.0543 - acc: 0.9914 - val_loss: 0.0169 - val_acc: 1.0000
Epoch 4/10
582/582 [==============================] - 36s 61ms/step - loss: 0.0883 - acc: 0.9794 - val_loss: 0.0147 - val_acc: 1.0000
Epoch 5/10
582/582 [==============================] - 36s 62ms/step - loss: 0.0507 - acc: 0.9897 - val_loss: 0.0141 - val_acc: 1.0000
Epoch 6/10
582/582 [==============================] - 28s 49ms/step - loss: 0.0474 - acc: 0.9931 - val_loss: 0.0139 - val_acc: 1.0000
Epoch 7/10
582/582 [==============================] - 27s 47ms/step - loss: 0.0591 - acc: 0.9914 - val_loss: 0.0156 - val_acc: 1.0000
Epoch 8/10
582/58

In [24]:
model.save('shizz_69_86.h5')

In [6]:
encoder = load_model('encoder.h5')
get_encoding = K.function([encoder.layers[0].input],
                                  [encoder.layers[6].output])
def encode(X):
    X = X.astype('float32') / 255.    
    return get_encoding([X])[0]

def predict(model,X,verbose=True):
    pred = model.predict(X)
    max_pred = [np.argmax(i) for i in pred]
    if verbose:
        print("Max Preds time", max_pred)
    counts = np.bincount(max_pred)
    class_pred = np.argmax(counts)
    return class_pred

def evaluate(model, X_test,Y_test,verbose = True):
    count = 0
    preds = []
    for i in range(len(X_test)):
        class_pred = predict(model,X_test[i],verbose=verbose)
        preds.append(class_pred)
        actual = Y_test[i]
        if verbose:
            print("Pred",classes[class_pred],"Actual",classes[actual])
            print()
        if class_pred == actual:
            count += 1
    if verbose:
        print("Confusion Matrix")
        print(confusion_matrix(Y_test,preds))
    return float(count)/float(len(Y_test)) * 100.0

In [ ]:
X_test[0].shape

In [26]:
evaluate(model, X_test, Y_test, verbose=True)

Max Preds time [0]
Pred Kicking Actual Kicking

Max Preds time [0]
Pred Kicking Actual Kicking

Max Preds time [0]
Pred Kicking Actual Kicking

Max Preds time [0]
Pred Kicking Actual Kicking

Max Preds time [0]
Pred Kicking Actual Kicking

Max Preds time [0]
Pred Kicking Actual Kicking

Max Preds time [7]
Pred Walking Actual Riding-Horse

Max Preds time [1, 1, 1]
Pred Riding-Horse Actual Riding-Horse

Max Preds time [3]
Pred SkateBoarding Actual Riding-Horse

Max Preds time [4, 4, 4]
Pred Swing-Bench Actual Riding-Horse

Max Preds time [7, 7, 7, 7]
Pred Walking Actual Running

Max Preds time [7, 7, 7, 7]
Pred Walking Actual Running

Max Preds time [7, 7, 8, 7]
Pred Walking Actual Running

Max Preds time [2, 2, 2, 2]
Pred Running Actual Running

Max Preds time [1, 1, 8, 8]
Pred Riding-Horse Actual SkateBoarding

Max Preds time [2, 6, 6, 2]
Pred Running Actual SkateBoarding

Max Preds time [3, 3, 2, 2]
Pred Running Actual SkateBoarding

Max Preds time [2, 2, 2, 2]
Pred Running Actual Ska

64.28571428571429

In [27]:
evaluate(model, X_test_unseen, Y_test_unseen, verbose=True)

Max Preds time [0]
Pred Kicking Actual Kicking

Max Preds time [0]
Pred Kicking Actual Kicking

Max Preds time [7]
Pred Walking Actual Kicking

Max Preds time [0]
Pred Kicking Actual Kicking

Max Preds time [1, 1, 1]
Pred Riding-Horse Actual Riding-Horse

Max Preds time [7]
Pred Walking Actual Riding-Horse

Max Preds time [8, 8, 7, 0]
Pred Golf-Swing Actual Running

Max Preds time [3, 3, 3, 3]
Pred SkateBoarding Actual SkateBoarding

Max Preds time [3, 3, 3, 3]
Pred SkateBoarding Actual SkateBoarding

Max Preds time [4, 4]
Pred Swing-Bench Actual Swing-Bench

Max Preds time [4, 4]
Pred Swing-Bench Actual Swing-Bench

Max Preds time [4, 4]
Pred Swing-Bench Actual Swing-Bench

Max Preds time [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Pred Lifting Actual Lifting

Max Preds time [6, 6, 6, 6, 6]
Pred Swing-Side Actual Swing-Side

Max Preds time [6]
Pred Swing-Side Actual Swing-Side

Max Preds time [7, 7, 7, 7, 7, 7, 7, 7]
Pred Walking Actual Walking

Max Preds time [7, 7, 7, 7, 7]
Pred Walking Actual W

86.36363636363636

In [ ]:
import time
start = time.time()
X_encoded = []
for i in range(len(X_test_unseen)):
    X_video = []
    for j in range(len(X_test_unseen[i])):
        X_video.append(encode(X_test_unseen[i][j]))
    X_video = np.array(X_video)
    print(X_video.shape)
    X_encoded.append(X_video)
print(time.time() - start)